<a href="https://colab.research.google.com/github/sofittech/hackathon-ml/blob/main/Hackthon_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [289]:
#Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore')

In [253]:
# Import training and test data
train = pd.read_csv('training-data.csv')
test = pd.read_csv('test-data.csv')

# DATES FEATURES
def date_features(df):
    # Date Features
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['day'] = df.date.dt.day
    df['dayofyear'] = df.date.dt.dayofyear
    df['dayofweek'] = df.date.dt.dayofweek
    df['weekofyear'] = df.date.dt.weekofyear
    
    # Additionnal Data Features
    df['day^year'] = np.log((np.log(df['dayofyear'] + 1)) ** (df['year'] - 2000))
    
    # Drop date
    df.drop('date', axis=1, inplace=True)
    
    return df

# Dates Features for Train, Test
train, test = date_features(train), date_features(test)

In [254]:
train.head()

,store,item code,sales,year,month,day,dayofyear,dayofweek,weekofyear,day^year
0,1,1,13,2016.0,1.0,1.0,1.0,4.0,53.0,-5.864207
1,1,1,11,2016.0,2.0,1.0,32.0,0.0,5.0,20.028234
2,1,1,14,2016.0,3.0,1.0,61.0,1.0,9.0,22.681333
3,1,1,13,2016.0,4.0,1.0,92.0,4.0,13.0,24.180730
4,1,1,10,2016.0,5.0,1.0,122.0,6.0,17.0,25.138418


In [255]:
daily_avg = train['daily_avg']  = train.groupby(['item code','store','dayofweek'])['sales'].transform('mean')
train['monthly_avg'] = train.groupby(['item code','store','month'])['sales'].transform('mean')
train = train.dropna()

# Average sales for Day_of_week = d per Item,Store
daymonth_avg = train.groupby(['item code','store','dayofweek'])['sales'].mean().reset_index()
# Average sales for Month = m per Item,Store
monthly_avg = train.groupby(['item code','store','month'])['sales'].mean().reset_index()

In [256]:

train.head()


,store,item code,sales,year,month,day,dayofyear,dayofweek,weekofyear,day^year,daily_avg,monthly_avg
0,1,1,13,2016.0,1.0,1.0,1.0,4.0,53.0,-5.864207,17.509579,16.025806
1,1,1,11,2016.0,2.0,1.0,32.0,0.0,5.0,20.028234,19.761538,16.333333
2,1,1,14,2016.0,3.0,1.0,61.0,1.0,9.0,22.681333,20.846743,19.212903
3,1,1,13,2016.0,4.0,1.0,92.0,4.0,13.0,24.180730,17.509579,20.286667
4,1,1,10,2016.0,5.0,1.0,122.0,6.0,17.0,25.138418,19.114943,21.838710


In [257]:
test.head()

,store,item code,year,month,day,dayofyear,dayofweek,weekofyear,day^year
0,1,1,2020,1,1,1,2,1,-7.330258
1,1,1,2020,2,1,32,5,5,25.035293
2,1,1,2020,3,1,61,6,9,28.351666
3,1,1,2020,4,1,92,2,14,30.225912
4,1,1,2020,5,1,122,4,18,31.423022


In [127]:
# def merge(df1, df2, col,col_name):
#   df1 =pd.merge(df1, df2, how='left', on=None, left_on=col, right_on=col,left_index=False, right_index=False, sort=True,copy=True, indicator=False)
#   df1 = df1.rename(columns={'sales':col_name})
#   return df1

# # Add Daily_avg and Monthly_avg features to test 
# # test = merge(test, daily_avg,['item','store','dayofweek'],['daily_avg'])
# # test = merge(test, monthly_avg,['item','store','month'],['monthly_avg'])

# # Sales Rolling mean sequence per item 
# rolling_10 = train.groupby(['item'])['sales'].rolling(10).mean().reset_index().drop('level_1', axis=1)
# train['rolling_mean'] = rolling_10['sales'] 

# # 90 last days of training rolling mean sequence added to test data
# rolling_last90 = train.groupby(['item','store'])['rolling_mean'].tail(90).copy()
# test['rolling_mean'] = rolling_last90.reset_index().drop('index', axis=1)

# # Shifting rolling mean 3 months
# train['rolling_mean'] = train.groupby(['item'])['rolling_mean'].shift(90) # Create a feature with rolling mean of day - 90
# train.head()

,store,item,sales,year,month,day,dayofyear,dayofweek,weekofyear,day^year,daily_avg,monthly_avg,rolling_mean
0,1,1,13,2013,1,1,1,1,1,-4.764668,18.168582,13.709677,NaN
1,1,1,11,2013,1,2,2,2,1,1.222622,18.793103,13.709677,NaN
2,1,1,14,2013,1,3,3,3,1,4.246245,19.452107,13.709677,NaN
3,1,1,13,2013,1,4,4,4,1,6.186505,21.015326,13.709677,NaN
4,1,1,10,2013,1,5,5,5,1,7.581575,22.973180,13.709677,NaN


In [258]:
test.head(20000)

,store,item code,year,month,day,dayofyear,dayofweek,weekofyear,day^year
0,1,1,2020,1,1,1,2,1,-7.330258
1,1,1,2020,2,1,32,5,5,25.035293
2,1,1,2020,3,1,61,6,9,28.351666
3,1,1,2020,4,1,92,2,14,30.225912
4,1,1,2020,5,1,122,4,18,31.423022
...,...,...,...,...,...,...,...,...,...
19995,3,23,2020,1,16,16,3,3,20.828230
19996,3,23,2020,1,17,17,4,3,21.227703
19997,3,23,2020,1,18,18,5,3,21.598366
19998,3,23,2020,1,19,19,6,3,21.943774


In [259]:
# Clean features highly correlated to each others
# for df in [train, test]:
train.drop(['day^year', 
              'weekofyear',
              'daily_avg',
              'day',
              'store',],
              axis=1, 
              inplace=True)
    


In [260]:
train.head()

,item code,sales,year,month,dayofyear,dayofweek,monthly_avg
0,1,13,2016.0,1.0,1.0,4.0,16.025806
1,1,11,2016.0,2.0,32.0,0.0,16.333333
2,1,14,2016.0,3.0,61.0,1.0,19.212903
3,1,13,2016.0,4.0,92.0,4.0,20.286667
4,1,10,2016.0,5.0,122.0,6.0,21.838710


In [261]:
test.drop(['day^year', 
              'weekofyear',
              'day',
              'store',],
              axis=1, 
              inplace=True)

In [262]:
test.head()

,item code,year,month,dayofyear,dayofweek
0,1,2020,1,1,2
1,1,2020,2,32,5
2,1,2020,3,61,6
3,1,2020,4,92,2
4,1,2020,5,122,4


In [84]:
# Features Scaling (except sales)
# sales_series, id_series = train['sales'], test['id']
# # Features Scaling
# train = (train - train.mean()) / train.std()
# test = (test - test.mean()) / test.std()
# # Retrieve actual Sales values and ID
# train['sales'] = sales_series
# test['id'] = id_series



In [263]:
train.head()

,item code,sales,year,month,dayofyear,dayofweek,monthly_avg
0,1,13,2016.0,1.0,1.0,4.0,16.025806
1,1,11,2016.0,2.0,32.0,0.0,16.333333
2,1,14,2016.0,3.0,61.0,1.0,19.212903
3,1,13,2016.0,4.0,92.0,4.0,20.286667
4,1,10,2016.0,5.0,122.0,6.0,21.838710


In [264]:
test.head()

,item code,year,month,dayofyear,dayofweek
0,1,2020,1,1,2
1,1,2020,2,32,5
2,1,2020,3,61,6
3,1,2020,4,92,2
4,1,2020,5,122,4


In [90]:
import xgboost as xgb

In [265]:
# Training Data
X_train = train.drop('sales', axis=1).dropna()
y_train = train['sales']
# Test Data
test.sort_values(by=['item code'], inplace=True)
# X_test = test.drop('item code', axis=1)
#df = train
df_train = train.copy()

# Train Test Split
X_train , X_test ,y_train, y_test = train_test_split(df_train.drop('sales',axis=1),df_train.pop('sales'), random_state=123, test_size=0.2)

# XGB Model
matrix_train = xgb.DMatrix(X_train, label = y_train)
matrix_test = xgb.DMatrix(X_test, label = y_test)

# Run XGB 
model = xgb.train(params={'objective':'reg:linear','eval_metric':'mae'}
                ,dtrain = matrix_train, num_boost_round = 500, 
                early_stopping_rounds = 20, evals = [(matrix_test,'test')],)

[00:01:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	test-mae:36.2639
Will train until test-mae hasn't improved in 20 rounds.
[1]	test-mae:25.4701
[2]	test-mae:18.3672
[3]	test-mae:14.0667
[4]	test-mae:11.6155
[5]	test-mae:10.2734
[6]	test-mae:9.53663
[7]	test-mae:9.11762
[8]	test-mae:8.7665
[9]	test-mae:8.56617
[10]	test-mae:8.42999
[11]	test-mae:8.31908
[12]	test-mae:8.15298
[13]	test-mae:8.04199
[14]	test-mae:7.93128
[15]	test-mae:7.84819
[16]	test-mae:7.69721
[17]	test-mae:7.57035
[18]	test-mae:7.47359
[19]	test-mae:7.43213
[20]	test-mae:7.37831
[21]	test-mae:7.27869
[22]	test-mae:7.20143
[23]	test-mae:7.17582
[24]	test-mae:7.13758
[25]	test-mae:7.11371
[26]	test-mae:7.09109
[27]	test-mae:7.04245
[28]	test-mae:7.02381
[29]	test-mae:6.95425
[30]	test-mae:6.91833
[31]	test-mae:6.90666
[32]	test-mae:6.86387
[33]	test-mae:6.84157
[34]	test-mae:6.78263
[35]	test-mae:6.74482
[36]	test-mae:6.68446
[37]	test-mae:

In [266]:
test.head()

,item code,year,month,dayofyear,dayofweek
0,1,2020,1,1,2
593,1,2020,2,54,6
594,1,2020,2,55,0
595,1,2020,2,56,1
596,1,2020,2,57,2


In [267]:
X_test.head()

,item code,year,month,dayofyear,dayofweek,monthly_avg
534104,30,2018.0,2.0,38.0,2.0,38.851064
592708,33,2018.0,12.0,355.0,4.0,44.845161
9456,1,2016.0,11.0,328.0,2.0,17.486667
639449,36,2016.0,12.0,351.0,4.0,58.800000
137180,8,2016.0,8.0,232.0,4.0,62.735484


In [268]:
matrix_test = xgb.DMatrix(X_test)

In [269]:
y_pred = model.predict(matrix_test)

In [270]:
predictions = [round(value) for value in y_pred]


In [271]:
print(predictions)

[59, 30, 18, 34, 42, 26, 21, 19, 85, 87, 83, 27, 41, 19, 45, 37, 36, 44, 25, 13, 30, 135, 71, 52, 31, 46, 51, 72, 87, 102, 51, 88, 50, 47, 48, 84, 20, 141, 64, 80, 25, 45, 103, 42, 54, 54, 39, 46, 28, 26, 25, 109, 56, 41, 87, 50, 21, 22, 35, 33, 44, 112, 49, 98, 58, 13, 28, 50, 45, 69, 42, 23, 25, 18, 116, 102, 24, 25, 64, 33, 106, 45, 44, 92, 60, 7, 35, 94, 86, 37, 70, 56, 32, 21, 52, 93, 43, 102, 65, 13, 32, 31, 19, 53, 84, 36, 45, 43, 22, 42, 52, 67, 40, 42, 74, 95, 40, 105, 80, 102, 122, 28, 77, 48, 44, 29, 29, 40, 61, 30, 39, 33, 17, 14, 43, 78, 35, 60, 28, 96, 106, 53, 22, 66, 45, 60, 48, 115, 15, 66, 35, 58, 40, 32, 30, 36, 52, 61, 67, 23, 23, 55, 19, 116, 97, 47, 57, 30, 64, 35, 74, 11, 27, 95, 107, 85, 53, 57, 81, 10, 43, 60, 29, 22, 19, 35, 50, 39, 31, 42, 49, 42, 103, 44, 21, 33, 88, 100, 68, 25, 28, 68, 30, 80, 46, 28, 69, 32, 66, 49, 54, 75, 47, 35, 29, 71, 65, 110, 85, 62, 88, 97, 69, 114, 22, 50, 17, 89, 87, 58, 21, 30, 87, 50, 23, 22, 26, 48, 108, 24, 51, 49, 58, 36, 42

In [272]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [274]:
model = pickle.load(open( "finalized_model.sav", "rb"))

[00:09:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [282]:
data = {"item code":8,"year": 2016, "month":8,	"dayofyear":232,	"dayofweek":4,	"monthly_avg":62.7}

In [283]:
df = pd.DataFrame(data, index = [0])

In [284]:
cols=["item code","year", "month",	"dayofyear",	"dayofweek",	"monthly_avg"]
df = df[cols]

In [285]:
df.head()

,item code,year,month,dayofyear,dayofweek,monthly_avg
0,8,2016,8,232,4,62.7


In [286]:
matrix_test = xgb.DMatrix(df)

In [287]:
sales = model.predict(matrix_test)

In [288]:
val = [round(value) for value in sales]
print(val[0])

42
